In [1]:
import pandas as pd
import numpy as np

In [2]:
# 対象国
countries = ["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"]

In [3]:
# Load data
pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

In [4]:
# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(countries)][['year', 'country', 'rgdpna', 'rkna', 'emp', 'labsh', 'pop', 'avh', 'rtfpna']]
# サンプル期間は1995年から2019年
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

In [5]:
# Calculate growth rates and Solow residual
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['a'] = 1 - data['labsh']  # Capital share

In [6]:
# Order by year
data = data.sort_values('year')

# Group by country
grouped_data = data.groupby('country')

In [7]:
# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capital

In [8]:
# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('country').agg({'g': 'mean',
                                       'a': "mean"})

In [9]:
# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['Capital Share'] = summary['a']
summary['Capital Deepening'] = summary['Growth Rate'] * summary['Capital Share']
summary['TFP Share'] = 1 - summary['Capital Share']
summary['TFP Growth'] = summary['Growth Rate'] * summary['TFP Share']

In [10]:
# Display output
pd.options.display.float_format = '{:.4f}'.format
display(summary[['Growth Rate', 'TFP Growth', 'Capital Deepening', 'TFP Share', 'Capital Share']])

,Growth Rate,TFP Growth,Capital Deepening,TFP Share,Capital Share
country,,,,,
Canada,0.8877,0.5804,0.3073,0.6538,0.3462
France,0.8292,0.5131,0.3161,0.6188,0.3812
Germany,0.6611,0.4160,0.2451,0.6293,0.3707
Italy,-0.0372,-0.0192,-0.0180,0.5152,0.4848
Japan,0.7110,0.4048,0.3061,0.5694,0.4306
United Kingdom,1.0155,0.5961,0.4194,0.5870,0.4130
United States,1.5370,0.9325,0.6044,0.6067,0.3933
